In [ ]:

import numpy as np

import pandas as pd

import os

# 현재 스크립트 위치 기준으로 상대 경로 설정
file_path = os.path.join(os.getcwd(), "data", "booksummaries.txt")

# 파일 읽기
with open(file_path, "r", encoding="utf-8") as file:
    data = file.readlines()

# 출력 예시
print(data[:1])  # 첫 5줄 출력



In [8]:
pip install nltk


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [11]:
import multiprocessing as mp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
import nltk
nltk.download('stopwords', quiet=True)

True

In [12]:
def preprocess_text(text):
    # lowercasing
    lowercased_text = text.lower()

    # cleaning 
    import re 
    remove_punctuation = re.sub(r'[^\w\s]', '', lowercased_text)
    remove_white_space = remove_punctuation.strip()

    # Tokenization = Breaking down each sentence into an array
    from nltk.tokenize import word_tokenize
    tokenized_text = word_tokenize(remove_white_space)

    # Stop Words/filtering = Removing irrelevant words
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words('english'))
    stopwords_removed = [word for word in tokenized_text if word not in stopwords]

    # Stemming = Transforming words into their base form
    from nltk.stem import PorterStemmer
    ps = PorterStemmer()
    stemmed_text = [ps.stem(word) for word in stopwords_removed]
    
    # Putting all the results into a dataframe.
    df = pd.DataFrame({
        'DOCUMENT': [text],
        'LOWERCASE' : [lowercased_text],
        'CLEANING': [remove_white_space],
        'TOKENIZATION': [tokenized_text],
        'STOP-WORDS': [stopwords_removed],
        'STEMMING': [stemmed_text]
    })

    return df

In [13]:
def calculate_tfidf(corpus):
    # Call the preprocessing result
    df = preprocessing(corpus)
        
    # Make each array row from stopwords_removed to be a sentence
    stemming = corpus['STEMMING'].apply(' '.join)
    
    # Count TF-IDF
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(stemming)
    
    # Get words from stopwords array to use as headers
    feature_names = vectorizer.get_feature_names_out()

    # Combine header titles and weights
    df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
    df_tfidf = pd.concat([df, df_tfidf], axis=1)

    return df_tfidf

In [14]:
def cosineSimilarity(corpus):
    # Call the TF-IDF result
    df_tfidf = calculate_tfidf(corpus)
    
    # Get the TF-IDF vector for the first item (index 0)
    vector1 = df_tfidf.iloc[0, 6:].values.reshape(1, -1)

    # Get the TF-IDF vector for all items except the first item
    vectors = df_tfidf.iloc[:, 6:].values
    
    # Calculate cosine similarity between the first item and all other items
    from sklearn.metrics.pairwise import cosine_similarity
    cosim = cosine_similarity(vector1, vectors)
    cosim = pd.DataFrame(cosim)
    
    # Convert the DataFrame into a one-dimensional array
    cosim = cosim.values.flatten()

    # Convert the cosine similarity result into a DataFrame
    df_cosim = pd.DataFrame(cosim, columns=['COSIM'])

    # Combine the TF-IDF array with the cosine similarity result
    df_cosim = pd.concat([df_tfidf, df_cosim], axis=1)

    return df_cosim

In [17]:
import os
import pandas as pd

# Define the column names
column_names = [
    'Wikipedia article ID', 
    'Freebase ID', 
    'Book title', 
    'Author', 
    'Publication date', 
    'Book genres (Freebase ID:name tuples)', 
    'Plot summary'
]

# 현재 작업 디렉토리에서 파일 경로 설정
file_path = os.path.join(os.getcwd(), "data", "booksummaries.txt")

# 파일이 존재하는지 확인 후 읽기
if os.path.exists(file_path):
    df = pd.read_csv(file_path, delimiter='\t', names=column_names, header=None)  # Adjust delimiter as needed

    # Save the DataFrame to a .csv file with column names
    output_path = os.path.join(os.getcwd(), "output.csv")
    df.to_csv(output_path, index=False)
    
    print(f"CSV 파일이 저장되었습니다: {output_path}")
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")


CSV 파일이 저장되었습니다: c:\Users\lissa\Desktop\2025\헤슬\책지피티\output.csv


In [20]:
file_path='./output.csv'
df = pd.read_csv(file_path)

In [26]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------------------- -------------- 7.1/11.1 MB 39.6 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 31.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   -------- ------------------------------- 8.9/43.6 MB 42.6 MB/s eta 0:00:01
   ---------------- ----------------------- 18.1/43.6 MB 42.2 MB/s eta 0:00:01
   --------------------- ------------------ 23.9/43.6 MB 37.7 MB/s eta 0:00:01
   ------------------------------ --------- 33.0/43.6 MB 38.8 MB/s eta 0:00:01
   ---------------------------------------  43.0/43.6 MB 40.2 MB/s eta 0:00:01
   ---------------------------------------- 43.6/43.6 MB 35.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Apply TF-IDF on the 'Plot summary' column
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Plot summary'])

# Show the shape of the resulting TF-IDF matrix
print(tfidf_matrix.shape)

(16559, 121260)


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Show the resulting similarity matrix
print(cosine_sim_matrix)

[[1.         0.1299163  0.18956561 ... 0.14520625 0.03338112 0.11606397]
 [0.1299163  1.         0.17445624 ... 0.13893065 0.02769185 0.11881149]
 [0.18956561 0.17445624 1.         ... 0.19229554 0.0465808  0.16297015]
 ...
 [0.14520625 0.13893065 0.19229554 ... 1.         0.03152995 0.13596954]
 [0.03338112 0.02769185 0.0465808  ... 0.03152995 1.         0.02981897]
 [0.11606397 0.11881149 0.16297015 ... 0.13596954 0.02981897 1.        ]]


In [29]:
def recommend_books(book_title, df, cosine_sim_matrix, top_n=5):
    # Reset the index of the DataFrame and create a reverse mapping of indices and book titles
    df = df.reset_index()
    indices = pd.Series(df.index, index=df['Book title']).drop_duplicates()
    
    # Check if the book exists in the dataset
    if book_title not in indices:
        print(f"Book titled '{book_title}' not found in the dataset.")
        return []
    
    # Get the index of the book that matches the title
    idx = indices[book_title]
    
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    
    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Skip the first book (itself) and get the scores of the top_n most similar books
    sim_scores = sim_scores[1:top_n+1]
    
    # Get the book indices
    book_indices = [i[0] for i in sim_scores]
    
    # Return the top_n most similar books
    return df['Book title'].iloc[book_indices].tolist()

In [30]:
# Example usage:
book_to_recommend = "Book of Joshua"
recommended_books = recommend_books(book_to_recommend, df, cosine_sim_matrix, top_n=5)

print(f"Books similar to '{book_to_recommend}':")
for idx, title in enumerate(recommended_books, 1):
    print(f"{idx}. {title}")

Books similar to 'Book of Joshua':
1. Book of Numbers
2. Veracity
3. Kaleidoscope Century
4. Joshua Then and Now
5. Fevre Dream
